In [ ]:
# Import necessary packages
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [ ]:
import seaborn as sbn
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [ ]:

# Import data into a dataframe and read it in our environment
sales = pd.DataFrame(pd.read_excel("UAM - Marketing Analytics - T05 Case study - Dataset.xlsx"))


In [ ]:
# Drop data that is zero or less quantity sold
sales = sales.drop(sales[sales["Quantity"] < 1].index)


In [ ]:
# Read the first 5 rows in our data
sales.head()

In [ ]:
# Set the current date as January 30, 2021
present = dt.datetime(2021,1,30)


In [ ]:
# Make sure that date is read as datatime format
sales['TransactionDate'] = pd.to_datetime(sales['TransactionDate'])


In [ ]:
# Group all sales (transactions) by customer to show number of days, number of transactions, and total revenue by customer
rfm = sales.groupby('CustomerID').agg({'TransactionDate': lambda date: (present -
date.max()).days, # Recency
'TransactionNo': lambda x: len(x), # Frequency
'Revenue': lambda x: x.sum()}) # Monetary Value

In [ ]:
# Let's rename columns FROM: TransactionDate, TransactionNo, and Revenue TO: Recency, Frequency, and MonetaryValue
rfm.rename(columns={'TransactionDate': 'Recency',
'TransactionNo': 'Frequency',
'Revenue': 'MonetaryValue'}, inplace=True)


In [ ]:
# Let's make sure that MonetaryValue values are defined as integers with NO decimal places
rfm['MonetaryValue'] = rfm['MonetaryValue'].astype(int)

In [ ]:
# See the data contained in new variables
rfm.head()


In [ ]:
# Review descriptive statistics for our RFM data
rfm.describe().round(2)

In [ ]:
# Let's review the correlation
rfm.corr()


In [ ]:
# Let's create a visual representation of the correlation matrix using a heat map
sbn.heatmap(rfm.corr(), annot=True);
plt.title("Correlation for RFM data");


In [ ]:
# Let's save YOUR RFM table to CSV on your computer. Change “Your_Name” in the code for your actual name
rfm.to_csv('RFM_Table.csv')


In [ ]:
# Let's set up a function to check the skewness of each of our variables. The new function will require two inputs: (1) RFM dataframe and (2) column name (Recency, Frequency, and MonetaryValue)
# Skew values capture skewness values for each column
# Skew test check if skewness is statistically significant
# plt.title control title of each distribution plots
# sbn.hisplot will plot our data
def check_skew(df, column):
 skew = stats.skew(df[column])
 plt.title('Distribution of ' + column)
 sbn.histplot(df[column], kde=True , stat='density')
 print("{}’s Skew: {:.2f}".format(column, skew))
 return

In [ ]:
# Plot all 3 graphs togheter for a visual summary of distribution and skewness
plt.figure(figsize=(9, 9))
plt.subplot(3, 1, 1)
check_skew(rfm, 'Recency')
plt.subplot(3, 1, 2)
check_skew(rfm, 'Frequency')
plt.subplot(3, 1, 3)
check_skew(rfm, 'MonetaryValue')
plt.tight_layout()

In [ ]:
# Copy the RFM data to new dataframe so we can perform data log data transformation
rfm_log = rfm.copy()

In [ ]:
# Transform skewed data with log transformation
df_rfm_log = np.log(rfm_log)


In [ ]:
# Check for skewness after log transformation
plt.figure(figsize=(9, 9))
plt.subplot(3,1,1)
check_skew(df_rfm_log,'Recency')
plt.subplot(3,1,2)
check_skew(df_rfm_log,'Frequency')
plt.subplot(3,1,3)
check_skew(df_rfm_log,'MonetaryValue')
plt.tight_layout()

In [ ]:
# Note that the values are on different scales
df_rfm_log.describe().round(2)

In [ ]:
#Let's make sure all data is on the same scale before we run the clustering algorithm
scaler = StandardScaler()
#Let's run the preprocessing library standard scaler
scaler.fit(df_rfm_log)
#Scale all logged-transformed data
df_rfm_normal = scaler.transform(df_rfm_log)
#Store the new transformed RFM into a data frame
df_rfm_normal = pd.DataFrame(df_rfm_normal, index=df_rfm_log.index, columns=df_rfm_log.columns)


In [ ]:
# Check results after running the Standard Scaler
df_rfm_normal.describe().round(2)

In [ ]:
# Let's use a function that calculates the optimal number of clusters for K-Means
# Using our dataset, we will start with 2 clusters and try up to 11 clusters
def optimal_kmeans(df, start=2, end=11):
 #Create empty lists to save values needed to plot graphs
 n_clu = []
 km_ss = []
 wss = []

 # Create a loop to find optimal n_clusters
 for n_clusters in range(start, end):

 # Create cluster labels
   kmeans = KMeans(n_clusters=n_clusters)
   labels = kmeans.fit_predict(df)

 # Review model performance using silhouette_avg and inertia_score
   silhouette_avg = round(silhouette_score(df, labels, random_state=1), 3)
   wss_score = round(kmeans.inertia_, 2)

 # Add score to lists created earlier
   km_ss.append(silhouette_avg)
   n_clu.append(n_clusters)
   wss.append(wss_score)

 # Print n_clusters, silhouette_avg, and inertia_score
   print("No. clusters: {}, Silhouette Score: {}, Within-cluster Sum-of-squares: {}".format(
   n_clusters,
   silhouette_avg,
  wss_score))

#Plot two graphs at the end of the loop: Within-cluster sum-of-squares and Silhouette score
 if n_clusters == end - 1:
  plt.figure(figsize=(9,6))
  plt.subplot(2,1,1)
  plt.title("Within-Cluster Sum-of-Squares")
  sbn.pointplot(x=n_clu, y=wss)
  plt.subplot(2,1,2)
  plt.title("Silhouette Score")
  sbn.pointplot(x=n_clu, y=km_ss)
  plt.tight_layout()
  plt.show()

In [ ]:
# Import warnings filter
from warnings import simplefilter
# Ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
# Execute the K-Means function built and ran in previous step
optimal_kmeans(df_rfm_normal)


In [ ]:
# Now we apply the k-means cluster algorithm to two cluster sizes: k=3 and k=4
def kmeans(normalized_df, clusters_number, original_df):

 # Implement k-means clustering on normalized RFM dataset
 kmeans = KMeans(n_clusters = clusters_number, random_state = 1)
 kmeans.fit(normalized_df)

 # Extract cluster labels
 cluster_labels = kmeans.labels_

 # Create a cluster label column in original dataset
 df_new = original_df.assign(Cluster = cluster_labels)

 return df_new

In [ ]:
# Calculate average RFM values and size for each cluster
def rfm_values(df):
 df_new = df.groupby(['Cluster']).agg({
 'Recency': 'mean',
 'Frequency': 'mean',
 'MonetaryValue': ['mean', 'count']
 }).round(0)
 return df_new

In [ ]:
#Lets apply both functions from previous steps to normalized data for K=3 clusters
df_rfm_k3 = kmeans(df_rfm_normal, 3, rfm)
rfm_values(df_rfm_k3)

In [ ]:
#Lets apply both functions from previous steps to normalized data for K=4 clusters
df_rfm_k4 = kmeans(df_rfm_normal, 4, rfm)
rfm_values(df_rfm_k4)

In [ ]:
#Let’s create a line plot visualization of our clusters (k=3 and k=4) over RFM characteristics
def line_plot(normalised_df_rfm, df_rfm_kmeans, df_rfm_original):
 # Transform dataframe and line plot
 normalised_df_rfm = pd.DataFrame(normalised_df_rfm,
 index=rfm.index,
 columns=rfm.columns)
 normalised_df_rfm['Cluster'] = df_rfm_kmeans['Cluster']

 # Melt data into long format
 df_melt = pd.melt(normalised_df_rfm.reset_index(),
 id_vars=['CustomerID', 'Cluster'],
 value_vars=['Recency', 'Frequency', 'MonetaryValue'],
 var_name='Category',
 value_name='Value')
 plt.xlabel('Category')
 plt.ylabel('Value')
 sbn.pointplot(data=df_melt, x='Category', y='Value', hue='Cluster')
 return

In [ ]:
#Let’s visualize our graphs for 3 and 4 clusters by applying the function we built in the previousstep
plt.figure(figsize=(9,9))
plt.subplot(3,1,1)
plt.title('K=3 Clustering over RFM characteristics')
line_plot(df_rfm_normal, df_rfm_k3, rfm)
plt.subplot(3,1,2)
plt.title('K=4 Clustering over RFM characteristics')
line_plot(df_rfm_normal, df_rfm_k4, rfm)
plt.tight_layout()